<img src="./images/QIBM-logo.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

## _*Multi-qubit state tomography*_ <br />with the IBM Quantum Experience Web API 
© IBM Research

The latest version of this notebook is available on https://github.ibm.com/QuantumSDK/qx-sdk/scripts/.

For more information about how to use the Quantum Experience consult the [IBM Quantum Experience tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0) or check-out the [community](https://quantumexperience.ng.bluemix.net/qstage/#/community).

***
### Contributors
Andrew Cross

## Introduction



In [15]:
import copy

from IBMQuantumExperience import IBMQuantumExperience
import Qconfig
import Qhelpers.misc as misc

api = IBMQuantumExperience.IBMQuantumExperience(Qconfig.APItoken,Qconfig.config)

In [16]:
# QASM source to prepare the input state
qasm = """
OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
h q[0];
h q[2]; s q[2];
h q[4]; u1(pi/4) q[4];
"""
# A list of qubits for multi-qubit state tomography
tomo_qubits = [0,1,2,4]
# Device to use
device = "real"  # use "simulator" or "real"
shots = 1024
maxCredits = 3
input_qreg = 'q'
input_creg = 'c'

The ideal POVM elements are $\{ |k\rangle\langle k| \}_{k=0}^{2^n-1}$ but the measurement actually implements $\{ \Pi_k \}$ where each $\Pi_k$ is assumed to be diagonal and $\sum_k \Pi_k=I$. The diagonal assumption models assignment errors, i.e. I should have observed $k$ but I observed $k'$ instead. If we assume that all of the non-ideality comes from imperfect measurement, rather than state preparation and single qubit gate error, we can estimate the $\Pi_k$ in the following way. Suppose we prepare $|j\rangle\langle j|$ and measure. The probability of observing outcome $k$ is $\mathrm{Tr}(\Pi_k|j\rangle\langle j|)=\langle j|\Pi_k|j\rangle$.

In [29]:
zerostate="""
OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
"""
meas_str = "measure %s[%d] -> %s[%d];"
num_states = 2**len(tomo_qubits)
job = []
for j in range(num_states):
    c = [tomo_qubits[i] for i in filter(lambda x:(j&(1<<x))!=0,range(len(tomo_qubits)))]
    qasm = {'qasm': copy.copy(zerostate)}
    for q in c:
        qasm['qasm'] += "x %s[%d];\n" % (input_qreg, q)
    for q in tomo_qubits:
        qasm['qasm'] += meas_str % (input_qreg, q, input_creg, q)
    job.append(qasm)

The next line generates the QASM source for the job and submits it to the Quantum Experience.

In [19]:
# Submit the job
out = api.runJob(job, device, shots, maxCredits)

The next line waits for the job to finish.

In [20]:
res = misc.waitForJobs([out['id']], api)

status = {'RUNNING': 1} (0 seconds)
status = {'RUNNING': 1} (5 seconds)
status = {'RUNNING': 1} (10 seconds)
status = {'RUNNING': 1} (15 seconds)
status = {'RUNNING': 1} (20 seconds)
status = {'COMPLETED': 1} (25 seconds)


In [28]:
misc.getData(res[0],0)

{'00000': 949, '00010': 55, '00100': 2, '10000': 16, '10010': 2}

In [39]:
import numpy as np
Pis = []
for k in range(num_states):
    Pis.append(np.zeros((num_states,num_states)))
for j in range(num_states):
    for key, val in misc.getData(res[0],j).items():
        k = 0
        for i in tomo_qubits:
            if key[-1-i] == '1':
                k += 2**tomo_qubits.index(i)
        # take only bits of key in tomo_qubits
        Pis[k][j,j] = float(val)/float(shots)

We measure a complete set of observables corresponding to the non-identity Pauli elements on qubits in tomo_qubits. We can construct the Pauli $Z$ operators as sums of projectors $|k\rangle\langle k|$ with $\pm 1$ coefficients. The actual observables are sums of $\Pi_k$.